In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:96% !important; }</style>"))

In [2]:
# start a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()

## Load the data

In [3]:
# Load the data
data = spark.read.csv("data/walmart_stock.csv", inferSchema=True, header=True)

## What does the Schema look like?

In [4]:
data.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



## Print out the first 5 columns.

In [5]:
print (data.head(5))

[Row(Date=datetime.datetime(2012, 1, 3, 0, 0), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996), Row(Date=datetime.datetime(2012, 1, 4, 0, 0), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475), Row(Date=datetime.datetime(2012, 1, 5, 0, 0), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539), Row(Date=datetime.datetime(2012, 1, 6, 0, 0), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922), Row(Date=datetime.datetime(2012, 1, 9, 0, 0), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)]


## Use describe() to learn about the DataFrame.

In [6]:
data.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

## Format the mean and stddev to just show up to two decimal places. 

In [7]:
from pyspark.sql.functions import format_number

In [8]:
results = data.describe()

In [9]:
results.select('summary',
               format_number(results['Open'].cast('double'),2).alias('Open'),
               format_number(results['High'].cast('double'),2).alias('High'),
               format_number(results['Low'].cast('double'),2).alias('Low'),
               format_number(results['Close'].cast('double'),2).alias('Close'),
               results['Volume'].cast('int').alias('Volume')
              ).show()

+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



## Calculate the ratio of High Price versus volume of stock traded for a day

In [10]:
data.withColumn('HV Ratio', data['High']/data['Volume']).select('HV Ratio').show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



## What is the mean of the Close column?

In [11]:
from pyspark.sql.functions import mean

In [12]:
data.select(format_number(mean('Close')\
                          .cast('double'),2)\
            .alias('avg_close')).show()

+---------+
|avg_close|
+---------+
|    72.39|
+---------+



## What is the max and min of the Volume column?

In [13]:
from pyspark.sql.functions import min, max

In [14]:
data.select(min('Volume').alias('min_volume'),
            max('Volume').alias('max_volume')
           ).show()

+----------+----------+
|min_volume|max_volume|
+----------+----------+
|   2094900|  80898100|
+----------+----------+



## How many days was the Close lower than 60 dollars?

In [15]:
# using SQL syntax
data.filter('Close < 60').count()

81

In [16]:
# using python/dataframe syntax
from pyspark.sql.functions import count
result = data.filter(data['Close'] < 60)
result.select(count('Close').alias('close_count')).show()

+-----------+
|close_count|
+-----------+
|         81|
+-----------+



## What percentage of the time was the High greater than 80 dollars ?

In other words, (Number of Days High>80)/(Total Days in the dataset)


In [17]:
total = data.count()
high_gt80 = data.filter('High > 80').count()*1.0

100.0 * (high_gt80/total)

9.141494435612083

## What is the Pearson correlation between High and Volume?

[Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [18]:
from pyspark.sql.functions import corr

In [19]:
data.select(corr('High','Volume').alias('high_vol_corr')).show()

+-------------------+
|      high_vol_corr|
+-------------------+
|-0.3384326061737161|
+-------------------+



## What is the max High per year?

In [20]:
from pyspark.sql.functions import year

In [21]:
data.select(year('Date').alias('year'),'High').groupBy('year').max('High').show()

+----+---------+
|year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



## What is the average Close for each Calendar Month?

In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months.

In [22]:
from pyspark.sql.functions import month

In [23]:
data.select(month('Date').alias('month'),'Close').groupBy('month').mean('Close').orderBy('month').show()

+-----+-----------------+
|month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

